In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def create_cnn_model(input_shape, num_conv_layers, num_filters, filter_sizes, pool_sizes,
                     num_dense_layers, dense_units, dropout_rate, optimizer, loss):
    """
    Creates a convolutional neural network (CNN) model with specified parameters.

    Parameters:
        input_shape (tuple): shape of input data in the form (height, width, channels)
        num_conv_layers (int): number of convolutional layers
        num_filters (list): number of filters for each convolutional layer
        filter_sizes (list): size of filters for each convolutional layer
        pool_sizes (list): size of pooling windows for each pooling layer
        num_dense_layers (int): number of dense layers
        dense_units (list): number of units for each dense layer
        dropout_rate (float): rate of dropout regularization
        optimizer: optimizer function for training the model
        loss: loss function for training the model

    Returns:
        A compiled CNN model
    """

    # Define input layer
    input_layer = Input(shape=input_shape)

    # Add convolutional layers
    x = input_layer
    for i in range(num_conv_layers):
        x = Conv2D(num_filters[i], filter_sizes[i], padding='same', activation='relu')(x)
        x = MaxPooling2D(pool_sizes[i])(x)

    # Flatten output of last convolutional layer
    x = Flatten()(x)

    # Add dense layers
    for i in range(num_dense_layers):
        x = Dense(dense_units[i], activation='relu')(x)
        x = Dropout(dropout_rate)(x)

    # Add output layer for regression
    output_layer_reg = Dense(1, activation='linear', name='regression')(x)

    # Add output layer for classification
    output_layer_class = Dense(10, activation='softmax', name='classification')(x)

    # Define model
    model = Model(inputs=input_layer, outputs=[output_layer_reg, output_layer_class])

    # Compile model
    model.compile(optimizer=optimizer, loss=loss, metrics=['mse', 'accuracy'])

    return model

In [ ]:
def create_transfer_learning_model(model_name, input_shape, num_classes, num_layers_to_freeze):
    # Load pre-trained model with weights trained on ImageNet
    if model_name == 'vgg16':
        base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    elif model_name == 'vgg19':
        base_model = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
    elif model_name == 'resnet50':
        base_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    elif model_name == 'resnet101':
        base_model = ResNet101(include_top=False, weights='imagenet', input_shape=input_shape)
    elif model_name == 'resnet152':
        base_model = ResNet152(include_top=False, weights='imagenet', input_shape=input_shape)
    elif model_name == 'inceptionv3':
        base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    elif model_name == 'inceptionresnetv2':
        base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
    elif model_name == 'xception':
        base_model = Xception(include_top=False, weights='imagenet', input_shape=input_shape)
    elif model_name == 'mobilenetv2':
        base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
    elif model_name == 'efficientnetb0':
        base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)
    else:
        raise ValueError('Invalid model name. Please choose from: vgg16, vgg19, resnet50, resnet101, resnet152, '
                         'inceptionv3, inceptionresnetv2, xception, mobilenetv2, efficientnetb0')

    # Freeze layers
    for layer in base_model.layers[:num_layers_to_freeze]:
        layer.trainable = False

    # Add custom classification head
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)

    # Define model
    model = Model(inputs=base_model.input, outputs=x)

    return model

In [ ]:
def create_custom_cnn(input_shape, num_classes, num_regression_targets):
    # Input layer
    inputs = Input(shape=input_shape)

    # Convolutional layers
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    # Flatten layer
    x = Flatten()(x)

    # Dense layers for classification task
    x_classification = Dense(512, activation='relu')(x)
    x_classification = Dropout(0.5)(x_classification)
    x_classification = Dense(num_classes, activation='softmax', name='classification_output')(x_classification)

    # Dense layers for regression task
    x_regression = Dense(512, activation='relu')(x)
    x_regression = Dropout(0.5)(x_regression)
    x_regression = Dense(num_regression_targets, name='regression_output')(x_regression)

    # Define model
    model = Model(inputs=inputs, outputs=[x_classification, x_regression])

    return model

In [ ]:
# Define input shape
input_shape = (256, 256, 3)

# Create custom CNN model
model_cnn = create_cnn_model(input_shape=input_shape,
                             num_conv_layers=3,
                             num_filters=[32, 64, 128],
                             filter_sizes=[3, 3, 3],
                             pool_sizes=[2, 2, 2],
                             num_dense_layers=2,
                             dense_units=[64, 10],
                             dropout_rate=0.5,
                             optimizer=Adam(),
                             loss='mse')

# Create transfer learning model based on VGG16 architecture
model_transfer = create_transfer_learning_model(model_name='vgg16',
                                                 input_shape=input_shape,
                                                 num_classes=10,
                                                 num_layers_to_freeze=15)

# Compile models
model_cnn.compile(optimizer=Adam(), loss='mse', metrics=['mae'])
model_transfer.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Generate dummy data
import numpy as np

X_train = np.random.rand(100, 256, 256, 3)
y_train_regression = np.random.rand(100, 10)
y_train_classification = np.random.randint(0, 10, size=(100,))

# Train models
es = EarlyStopping(patience=3, verbose=1)

model_cnn.fit(X_train, y_train_regression, epochs=10, validation_split=0.2, callbacks=[es])
model_transfer.fit(X_train, y_train_classification, epochs=10, validation_split=0.2, callbacks=[es])